# Tobias fly OTU clustering

Let's get some reasonable species-level hypotheses for Tobias' fly sequences. We'll use the [usearch](#http://drive5.com/usearch/) family of clustering algorithms and the [uparse pipeline](#http://drive5.com/usearch/manual/uparse_pipeline.html) for creating an OTU table, specifically a biom table, for our downstream analyses. Usearch comes in two sizes, 64 and 32 bit. The 32 bit version is free and fine for small datasets, we can get it [here](3http://www.drive5.com/usearch/download.html). 

So let's download usearch, rename it and put it in our binaries (mac users are going to have to put it somewhere more local). Then get Bitty's fasta file, rename and turn it into a read-only file so we don't accidentally change it while using it. 

In [1]:
cd /home/daniel/Documents/tobias_flies

In [3]:
sudo mv usearch9.2.64_i86linux32 /usr/bin/usearch92

: 1

In [3]:
## for some reason, I don't have execution privileges on this...
chmod u+x usearch92

In [5]:
## get the fasta files, write protect them:
## renamed Bitty's latest fly file: "fasta.fas.txt" to "tobias_flies_edited_25.01.2106.fasta"

chmod 444 tobias_flies_edited_25.01.2106.fasta

In [6]:
## what do they look like?
head tobias_flies_edited_25.01.2106.fasta -n 4

>RLCI001-16|Staphylinidae|C10.08
CATAAAGATATTGGAACTTTATATTTTATTTTTGGCGCTTGATCTGGAATAGTAGGAACTTCTTTAAGATTATTAATCCGTACAGAATTAGGTAATCCAGGATCATTAATTGGAGATGATCAAATTTATAATGTTATTGTTACAGCCCATGCATTTATTATAATTTTCTTTATAGTTATACCAACCATAATTGGAGGATTTGGTAATTGATTAGTACCTTTAATATTAGGAGCACCTGATATAGCTTTCCCTCGAATAAATAATATAAGATTTTGACTTCTACCTCCTTCTCTATCTCTTCTATTAATAAGAAGCTTAATTGAAAGAGGAGCTGGAACTGGTTGAACAGTTTATCCCCCTCTTTCATCTAATATCGCCCATGGTGGAGCTTCTGTAGATACTGCAATTTTTAGATTACATTTAGCTGGAATTTCATCTATTTTAGGAGCAGTAAATTTTATTACTACAGTAATTAATATACGATCTACTGGAATATCTTTTGATCGTATATCTTTATTTATTTGATCAGTATCAATTACTGCATTACTTTTATTATTATCCTTGCCTGTACTAGCAGGAGCTATCACTATACTTTTAACTGACCGTAATCTTAATACTTCATTTTTTGATCCTGCTGGAGGAGGAGACCCTGTTTTATACCAACACTTATTTTGATTTTTTGGTCC
>RLCI003-16|Hirtodrosophila sp_10DG|C208.01
CATAAAGATATTGGAACTTTATATTTCATTTTTGGTGCTTGAGCAGGAATAGTGGGAACATCTTTAAGAATCTTAATTCGAGCTGAATTAGGACATCCAGGAGCTCTTATTGGTGATGACCAAATTTATAATGTAATTGTAACAGCCCATGCTTTTATTATAATTTTTTTTATAGTTATACCAGTAATAATTGGAGGATTTGGAAATTGACTTGTTCCCTTAATATTAGGAGCCCC

How many sequences are there?

In [7]:
grep "^>" tobias_flies_edited_25.01.2106.fasta | wc -l

615


Check these for primers. Tobias and Bitty used the barcode gene for [Cytochrome C oxidase](#https://en.wikipedia.org/wiki/Cytochrome_c_oxidase_subunit_I), or COI. The primers are found [here](http://www.dnabarcodes2011.org/conference/preconference/Smithsonian-LABprimertable.pdf), under named LCO1490 (forward) and HCO2198 (reverse). Bitty created consensus sequences in the forward direction (forward primer read in 5'-3'). We need to remove primers before clustering, if present. To look for them: 

In [8]:
## COI forward primers:
coiF=GGTCAACAAATCATAAAGATATTGG

## COI reverse primers, reverse compliment
coiRrc=TGATTTTTTGGTCACCCTGAAGTTTA

Are these forward primers in our sequences? We can look with grep.

In [9]:
grep .*$coiF tobias_flies_edited_25.01.2106.fasta -B 1 | head -n 5

>RLCI054-16|Zygothrica sp_41DG|C240.30
TGGTCAACAAATCATAAAGATATTGGAACATTATATTTTATTTTTGGTGCATGAGCTGGGATAGTCGGAACATCATTAAGAATCTTAATTCGAGCAGAATTAGGTCACCCAGGAGCCCTAATTGGTGATGATCAAATTTATAACGTAATTGTGACTGCTCATGCTTTTATTATAATTTTTTTTATAGTTATACCAATTATAATTGGTGGATTTGGAAATTGATTAGTTCCCTTAATATTAGGAGCTCCTGATATAGCATTTCCTCGAATAAATAATATAAGTTTCTGATTACTTCCTCCAGCTTTAACTCTTTTATTGGTTAGAAGAATAGTGGAAAATGGAGCTGGAACTGGATGAACAGTTTACCCTCCTTTATCATCAGGAATTGCTCATGGTGGAGCTTCTGTTGATCTAGCCATTTTTTCTCTTCATCTAGCAGGTATTTCATCAATTTTAGGAGCTGTAAATTTTATCACAACAGTAATTAATATACGATCATCTGGGATTACTCTTGATCGAATACCTTTATTTGTATGATCAGTTGTTATTACAGCTTTACTTCTTCTTTTATCTTTACCAGTTTTAGCAGGTGCTATTACTATATTATTAACAGATCGAAATTTAAATACTTCATTTTTTGACCCAGCTGGAGGAGGAGACCCAATTTTATACCAACATTTATTCTGATTTTTTGGTCACCCTGAAGTTTA
--
>RLCI056-16|Zygothrica sp_41DG|C240.32
TGGTCAACAAATCATAAAGATATTGGAACATTATATTTTATTTTTGGTGCATGAGCTGGGATAGTCGGAACATCATTAAGAATCTTAATTCGAGCAGAATTAGGTCACCCAGGAGCCCTAATTGGTGATGATCAAATTTATAACGTAATTGTAACTGCTCATGCTTTTATTATAATTTTTTTTATAGTTATACCAATTATAATTGGTG

Etc, etc. Yep, they're in there. How many?

In [10]:
grep .*$coiF tobias_flies_edited_25.01.2106.fasta | wc -l

12


How many contain reverse primers? Using the reverse compliment:

In [11]:
grep $coiRrc.*$ tobias_flies_edited_25.01.2106.fasta | wc -l

9


So let's remove these with SED:

In [12]:
sed s/.*$coiF//g tobias_flies_edited_25.01.2106.fasta | sed s/$coiRrc.*$//g > cleanflies.fasta

These are high-quality, hand-curated sanger sequences, so I'm going to skip a lot of the standard illumina quality checks, such as extensive testing for chimeric sequences, and removing singletons. 

The next step is to stack these reads into piles of identical (100% similarity, equal base pairs) sequences:

In [13]:
usearch92 -fastx_uniques cleanflies.fasta -fastaout tobias_uniques.fasta -sizeout -relabel Uniq

## if it turns out we have reverse sequences, include -strand both in this 

usearch v9.2.64_i86linux32, 3.9Gb RAM, 4 cores
(C) Copyright 2013-16 Robert C. Edgar, all rights reserved.
http://drive5.com/usearch

License: danchurchthomas@gmail.com

00:00 41Mb    100.0% Reading cleanflies.fasta
00:00 33Mb    100.0% DF                      
00:00 35Mb   615 seqs, 358 uniques, 293 singletons (81.8%)
00:00 35Mb   Min size 1, median 1, max 24, avg 1.72
00:00 35Mb    100.0% Writing tobias_uniques.fasta


Sort them, not going to use a minimum size. This is normally how we exclude singletons and I do not want to do this here:

In [14]:
usearch92 -sortbysize tobias_uniques.fasta -fastaout tobias_sorted.fasta -minsize 1

usearch v9.2.64_i86linux32, 3.9Gb RAM, 4 cores
(C) Copyright 2013-16 Robert C. Edgar, all rights reserved.
http://drive5.com/usearch

License: danchurchthomas@gmail.com

00:00 40Mb    100.0% Reading tobias_uniques.fasta
00:00 6.7Mb  Getting sizes                       
00:00 6.7Mb  Sorting 358 sequences
00:00 6.7Mb   100.0% Writing output


Now these look like this:

In [33]:
head tobias_sorted.fasta -n 15

>Uniq1;size=24;
CATAAAGATATTGGAACTTTATACTTTATTTTCGGTGCTTGAGCAGGAATAGTAGGGACATCTCTAAGAATTTTAATTCG
AGCTGAATTAGGTCATCCTGGTGCCTTAATTGGAGATGACCAAATTTATAACGTAATTGTTACTGCCCATGCTTTTGTAA
TAATTTTTTTTATAGTAATACCAATTATAATTGGAGGATTTGGAAACTGATTAGTGCCCCTAATGTTAGGAGCACCAGAT
ATAGCATTTCCTCGAATAAATAATATAAGTTTTTGATTACTACCTCCAGCTTTAACACTTTTATTGGTTAGAAGTATAGT
AGAAAATGGAGCTGGAACTGGATGAACAGTCTACCCACCTTTATCTGCAGGAATTGCCCACGGTGGAGCTTCAGTTGATT
TAGCAATTTTCTCATTACATTTAGCCGGAATTTCTTCAATTTTAGGAGCTGTAAATTTTATTACAACAGTAATTAATATA
CGATCATCAGGTATTACTCTTGATCGAATACCTTTATTTGTTTGATCAGTAGTTATTACTGCTTTACTTCTTCTTCTTTC
TTTACCAGTTTTAGCAGGAGCTATTACTATACTATTAACAGACCGAAATTTAAACACTTCATTTTTTGACCCAGCAGGAG
GAGGGGACCCTATTTTATACCAACATTTATTCTGATTTTTTGGTCA
>Uniq2;size=19;
CATAAAGATATTGGAACATTATATTTTATTTTTGGTGCCTGAGCAGGAATAGTGGGAACTTCTTTAAGAATTCTAATTCG
TGCTGAACTAGGACACCCAGGAGCTTTAATTGGAGATGATCAAATTTATAATGTGATTGTAACCGCTCATGCTTTCATTA
TAATTTTTTTTATAGTAATACCTATTATAATTGGAGGATTTGGGAATTGATTAGTCCCACTAATATTAGGAGCACCTGAT
ATAGCATTCCCACGAATAAATAATATAAGT

How many sets of unique sequences are there?

In [16]:
grep "^>" tobias_sorted.fasta | wc -l

358


Now cluster these into 99% similarity radius OTUs, using usearch. The [-cluster_smallmem](http://drive5.com/usearch/manual/cmd_cluster_smallmem.html) is the command of choice here, because we are using a radius other than 97%, and these are not illumina reads, see discussion [here](http://drive5.com/usearch/manual/uparse_otu_radius.html). 

In [17]:
usearch92 -cluster_smallmem tobias_sorted.fasta -sortedby size -id .99 -centroids tobias_99otus.fasta

usearch v9.2.64_i86linux32, 3.9Gb RAM, 4 cores
(C) Copyright 2013-16 Robert C. Edgar, all rights reserved.
http://drive5.com/usearch

License: danchurchthomas@gmail.com

00:00 43Mb    100.0% 85 clusters, max size 59, avg 4.2
00:00 44Mb    100.0% Writing centroids to tobias_99otus.fasta
                                                             
      Seqs  358
  Clusters  85
  Max size  59
  Avg size  4.2
  Min size  1
Singletons  41, 11.5% of seqs, 48.2% of clusters
   Max mem  44Mb
      Time  1.00s
Throughput  358.0 seqs/sec.



Let's also do a 98% similarity radius. This may be useful. 

In [18]:
usearch92 -cluster_smallmem tobias_sorted.fasta -sortedby size -id .98 -centroids tobias_98otus.fasta

usearch v9.2.64_i86linux32, 3.9Gb RAM, 4 cores
(C) Copyright 2013-16 Robert C. Edgar, all rights reserved.
http://drive5.com/usearch

License: danchurchthomas@gmail.com

00:00 43Mb    100.0% 79 clusters, max size 59, avg 4.5
00:00 43Mb    100.0% Writing centroids to tobias_98otus.fasta
                                                             
      Seqs  358
  Clusters  79
  Max size  59
  Avg size  4.5
  Min size  1
Singletons  32, 8.9% of seqs, 40.5% of clusters
   Max mem  43Mb
      Time  1.00s
Throughput  358.0 seqs/sec.



After this, we'll have duplicate lines of code for process, one for 98% and one for 99%. 

Looks like?

In [19]:
head tobias_99otus.fasta -n 15

>Uniq3;size=17;
CATAAAGATATTGGAACATTATATTTTATTTTTGGTGCATGAGCTGGGATAGTCGGAACATCATTAAGAATCTTAATTCG
AGCAGAATTAGGTCACCCAGGAGCCCTAATTGGTGATGATCAAATTTATAACGTAATTGTGACTGCTCATGCTTTTATTA
TAATTTTTTTTATAGTTATACCAATTATAATTGGTGGATTTGGAAATTGATTAGTTCCCTTAATATTAGGAGCTCCTGAT
ATAGCATTTCCTCGAATAAATAATATAAGTTTCTGATTACTTCCTCCAGCTTTAACTCTTTTATTGGTTAGAAGAATAGT
GGAAAATGGAGCTGGAACTGGATGAACAGTTTACCCTCCTTTATCATCAGGAATTGCTCATGGTGGAGCTTCTGTTGATC
TAGCCATTTTTTCTCTTCATCTAGCAGGTATTTCATCAATTTTAGGAGCTGTAAATTTTATCACAACAGTAATTAATATA
CGATCATCTGGGATTACTCTTGATCGAATACCTTTATTTGTATGATCAGTTGTTATTACAGCTTTACTTCTTCTTTTATC
TTTACCAGTTTTAGCAGGTGCTATTACTATATTATTAACAGATCGAAATTTAAATACTTCATTTTTTGACCCAGCTGGAG
GAGGAGACCCAATTTTATACCAACATTTATTCTGATTTTTTGGTCAC
>Uniq6;size=11;
CATAAAGATATTGGAACACTTTATTTTATTTTTGGGGTGTGGGCAGGAATAATTGGCACATCAATAAGAATACTTATTCG
TGCAGAATTAAGAACCCCCCTTCCTCTTTTAGGAAACGACCAAATTTTTAATGTTATTGTAACCGCTCATGCTTTTGTAA
TAATTTTTTTTATAGTAATACCTATTATAATGGGAGGATTTGGTAATTGACTAGTTCCATTAATACTAGGAGCCCCGGAT
ATAGCCTTCCCCCGAATAAATAATATAAG

In [20]:
head tobias_98otus.fasta -n 15

>Uniq3;size=17;
CATAAAGATATTGGAACATTATATTTTATTTTTGGTGCATGAGCTGGGATAGTCGGAACATCATTAAGAATCTTAATTCG
AGCAGAATTAGGTCACCCAGGAGCCCTAATTGGTGATGATCAAATTTATAACGTAATTGTGACTGCTCATGCTTTTATTA
TAATTTTTTTTATAGTTATACCAATTATAATTGGTGGATTTGGAAATTGATTAGTTCCCTTAATATTAGGAGCTCCTGAT
ATAGCATTTCCTCGAATAAATAATATAAGTTTCTGATTACTTCCTCCAGCTTTAACTCTTTTATTGGTTAGAAGAATAGT
GGAAAATGGAGCTGGAACTGGATGAACAGTTTACCCTCCTTTATCATCAGGAATTGCTCATGGTGGAGCTTCTGTTGATC
TAGCCATTTTTTCTCTTCATCTAGCAGGTATTTCATCAATTTTAGGAGCTGTAAATTTTATCACAACAGTAATTAATATA
CGATCATCTGGGATTACTCTTGATCGAATACCTTTATTTGTATGATCAGTTGTTATTACAGCTTTACTTCTTCTTTTATC
TTTACCAGTTTTAGCAGGTGCTATTACTATATTATTAACAGATCGAAATTTAAATACTTCATTTTTTGACCCAGCTGGAG
GAGGAGACCCAATTTTATACCAACATTTATTCTGATTTTTTGGTCAC
>Uniq6;size=11;
CATAAAGATATTGGAACACTTTATTTTATTTTTGGGGTGTGGGCAGGAATAATTGGCACATCAATAAGAATACTTATTCG
TGCAGAATTAAGAACCCCCCTTCCTCTTTTAGGAAACGACCAAATTTTTAATGTTATTGTAACCGCTCATGCTTTTGTAA
TAATTTTTTTTATAGTAATACCTATTATAATGGGAGGATTTGGTAATTGACTAGTTCCATTAATACTAGGAGCCCCGGAT
ATAGCCTTCCCCCGAATAAATAATATAAG

How many OTUs?

In [21]:
grep "^>" tobias_99otus.fasta | wc -l

85


In [22]:
grep "^>" tobias_98otus.fasta | wc -l

79


Now, if we want to get a membership list of these OTUs? As in, what fly sequences belong to each of these OTUs? It seems like there should be an easy way to get USEARCH to keep track as it goes of this, but I don't know it. Instead, I'll do a quick [BLAST](https://blast.ncbi.nlm.nih.gov/Blast.cgi) assignment, using our new OTUs as the reference database. First, let's cleanup the OTU labels a little:

In [23]:
sed 's/;size.*;/;/' tobias_99otus.fasta > tobias_99otus_relab.fasta
sed 's/;size.*;/;/' tobias_98otus.fasta > tobias_98otus_relab.fasta

Now the sequence headers look like this:

In [26]:
head -n 3 tobias_99otus_relab.fasta

>Uniq3;
CATAAAGATATTGGAACATTATATTTTATTTTTGGTGCATGAGCTGGGATAGTCGGAACATCATTAAGAATCTTAATTCG
AGCAGAATTAGGTCACCCAGGAGCCCTAATTGGTGATGATCAAATTTATAACGTAATTGTGACTGCTCATGCTTTTATTA


Make our BLAST-searchable database from our OTUs:

In [28]:
makeblastdb -in tobias_99otus_relab.fasta -dbtype nucl -logfile blastdb99_errors.txt
makeblastdb -in tobias_98otus_relab.fasta -dbtype nucl -logfile blastdb98_errors.txt

Now use this to query our individual sequences. Blast can spit out csv spreadsheets, and we can control which columns we want. Here I ask for the name of the sequence, the top OTU it blasted to, and the percent identity match. If we want to see other matches, "-max_target_seqs" can be adjusted. Visual inspection of second and third blast matches showed that they generally matched to one OTU much more strongly than any others, so I'll just keep the top matches, makes things easier. 

In [34]:
blastn -query tobias_flies_edited_25.01.2106.fasta -db tobias_99otus_relab.fasta -outfmt "10 qseqid sseqid pident" -out tobias_flies_OTU99memberships.csv -max_target_seqs 1
blastn -query tobias_flies_edited_25.01.2106.fasta -db tobias_98otus_relab.fasta -outfmt "10 qseqid sseqid pident" -out tobias_flies_OTU98memberships.csv -max_target_seqs 1

We can add a header:

In [35]:
sed '1 i\qseqid,sseqid,pident' tobias_flies_OTU99memberships.csv -i
sed '1 i\qseqid,sseqid,pident' tobias_flies_OTU98memberships.csv -i

In [36]:
head tobias_flies_OTU99memberships.csv

qseqid,sseqid,pident
RLCI001-16|Staphylinidae|C10.08,Uniq252,100.00
RLCI003-16|Hirtodrosophila,Uniq31,100.00
RLCI005-16|Neoponera,Uniq43,100.00
RLCI006-16|Neoponera,Uniq43,100.00
RLCI008-16|Zygothrica,Uniq21,99.85
RLCI010-16|Zygothrica,Uniq227,100.00
RLCI011-16|Laccodrosophila,Uniq27,100.00
RLCI012-16|Hirtodrosophila,Uniq32,99.70
RLCI013-16|Hirtodrosophila,Uniq25,100.00


In [37]:
head tobias_flies_OTU98memberships.csv

qseqid,sseqid,pident
RLCI001-16|Staphylinidae|C10.08,Uniq252,100.00
RLCI003-16|Hirtodrosophila,Uniq31,100.00
RLCI005-16|Neoponera,Uniq43,100.00
RLCI006-16|Neoponera,Uniq43,100.00
RLCI008-16|Zygothrica,Uniq21,99.85
RLCI010-16|Zygothrica,Uniq227,100.00
RLCI011-16|Laccodrosophila,Uniq27,100.00
RLCI012-16|Hirtodrosophila,Uniq32,99.70
RLCI013-16|Hirtodrosophila,Uniq25,100.00


And we can send this to Tobias and Bitty to see if it matches their previous results...